In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,4137
2,Huelva,Confirmados PDIA 14 días,1846
3,Huelva,Tasa PDIA 14 días,"361,43422425760116"
4,Huelva,Confirmados PDIA 7 días,943
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,23
629,Municipio de Huelva sin especificar,Total Confirmados,65
630,Municipio de Huelva sin especificar,Curados,15


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  4137.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1091.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 589 personas en los últimos 7 días 

Un positivo PCR cada 291 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,4137.0,943.0,1846.0,510743.0,184.632976,361.434224,226.0
Huelva-Costa,2083.0,454.0,897.0,288115.0,157.575968,311.334016,110.0
Condado-Campiña,1342.0,340.0,567.0,155057.0,219.274202,365.671979,86.0
Huelva (capital),1091.0,244.0,493.0,143663.0,169.841922,343.164211,62.0
Gibraleón,156.0,71.0,107.0,12607.0,563.179186,848.734830,32.0
Sierra de Huelva-Andévalo Central,647.0,126.0,348.0,67571.0,186.470527,515.013837,30.0
Lepe,317.0,74.0,140.0,27431.0,269.767781,510.371478,18.0
Almonte,222.0,79.0,131.0,24191.0,326.567732,541.523707,12.0
Aljaraque,158.0,42.0,69.0,21260.0,197.554092,324.553151,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),34.0,2.0,12.0,516.0,387.596899,2325.581395,0.0
Cortegana,115.0,9.0,85.0,4666.0,192.884698,1821.688813,3.0
Escacena del Campo,58.0,31.0,38.0,2284.0,1357.267951,1663.747811,8.0
Villarrasa,57.0,19.0,28.0,2176.0,873.161765,1286.764706,3.0
Almonaster la Real,23.0,3.0,20.0,1819.0,164.925783,1099.505223,3.0
Gibraleón,156.0,71.0,107.0,12607.0,563.179186,848.734830,32.0
Calañas,26.0,7.0,23.0,2791.0,250.806163,824.077392,3.0
Palma del Condado (La),190.0,39.0,77.0,10761.0,362.419849,715.546882,8.0
Valverde del Camino,120.0,40.0,86.0,12820.0,312.012480,670.826833,9.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cerro de Andévalo (El),27.0,1.0,10.0,2364.0,42.301184,423.011844,2.0,0.100000
Cortegana,115.0,9.0,85.0,4666.0,192.884698,1821.688813,3.0,0.105882
Almonaster la Real,23.0,3.0,20.0,1819.0,164.925783,1099.505223,3.0,0.150000
Granado (El),34.0,2.0,12.0,516.0,387.596899,2325.581395,0.0,0.166667
Villanueva de los Castillejos,30.0,1.0,6.0,2820.0,35.460993,212.765957,0.0,0.166667
Jabugo,19.0,1.0,5.0,2250.0,44.444444,222.222222,0.0,0.200000
Higuera de la Sierra,8.0,1.0,4.0,1287.0,77.700078,310.800311,0.0,0.250000
Zarza-Perrunal (La),4.0,1.0,4.0,1267.0,78.926598,315.706393,0.0,0.250000
Rosal de la Frontera,10.0,2.0,7.0,1697.0,117.855038,412.492634,0.0,0.285714
